#Streamlit app for final

In [1]:
pip install geopandas streamlit_folium plotly nbformat

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip freeze

altair==5.2.0
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
attrs==23.2.0
blinker==1.7.0
branca==0.7.2
cachetools==5.3.3
certifi==2024.2.2
charset-normalizer==3.3.2
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
colorama @ file:///home/conda/feedstock_root/build_artifacts/colorama_1666700638685/work
comm @ file:///home/conda/feedstock_root/build_artifacts/comm_1710320294760/work
contourpy==1.2.1
cycler==0.12.1
debugpy @ file:///D:/bld/debugpy_1707444547228/work
decorator @ file:///home/conda/feedstock_root/build_artifacts/decorator_1641555617451/work
exceptiongroup @ file:///home/conda/feedstock_root/build_artifacts/exceptiongroup_1704921103267/work
executing @ file:///home/conda/feedstock_root/build_artifacts/executing_1698579936712/work
fastjsonschema==2.19.1
fiona==1.9.6
folium==0.16.0
fonttools==4.51.0
geopandas==0.14.3
gitdb==4.0.11
GitPython==3.1.42
idna==3.6
importlib_metadata @ file:///home/conda/feedstock_root/build_artifacts/import

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt, matplotlib
import matplotlib.cm
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize
import plotly as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
import string, random
import seaborn as sns
import streamlit as st

pd.set_option('display.max_columns', None)

In [4]:
def load_data(csv):
    df = pd.read_csv(csv)
    return df

In [5]:
df = load_data('Data/gun-violence-data_01-2013_03-2018.csv')
df.dtypes

incident_id                      int64
date                            object
state                           object
city_or_county                  object
address                         object
n_killed                         int64
n_injured                        int64
incident_url                    object
source_url                      object
incident_url_fields_missing       bool
congressional_district         float64
gun_stolen                      object
gun_type                        object
incident_characteristics        object
latitude                       float64
location_description            object
longitude                      float64
n_guns_involved                float64
notes                           object
participant_age                 object
participant_age_group           object
participant_gender              object
participant_name                object
participant_relationship        object
participant_status              object
participant_type         

In [6]:
df['date']=pd.to_datetime(df['date'])
df['year']=df['date'].dt.year
df['month'] = df['date'].dt.month

In [7]:
df.head(2)

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,year,month
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN,2013,1
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,2013,1


GENDER

In [8]:
def count_genders(gender_str):
    genders = gender_str.split('||')
    male_count = 0
    female_count = 0
    for g in genders:
        if '::Male' in g:
            male_count += 1
        elif '::Female' in g:
            female_count += 1
    return male_count, female_count

# Apply the function to each row and sum the counts
gender_counts = df['participant_gender'].fillna('').apply(count_genders)
total_male_count = sum(count[0] for count in gender_counts)
total_female_count = sum(count[1] for count in gender_counts)

print("Total Male Count:", total_male_count)
print("Total Female Count:", total_female_count)

Total Male Count: 304103
Total Female Count: 42376


In [9]:
def separate(df):
    df=df.split("||")
    df=[(x.split("::")) for x in df]
    y = []
    for  i in range (0, len(df)):
        y.append(df[i][-1])
    return(y) 

In [10]:
df['participant_gender'] = df['participant_gender'].fillna("0::Zero")
df['gender'] = df['participant_gender'].apply(lambda x: separate(x))
df['Males'] = df['gender'].apply(lambda x: x.count('Male'))
df['Females'] = df['gender'].apply(lambda x: x.count('Female'))

In [11]:
gender_year_count = df[['year', 'Males', 'Females']].groupby('year').sum()
gender_year_count

,Males,Females
year,,
2013,981,283
2014,66809,9706
2015,68781,9821
2016,75707,10547
2017,75484,9781
2018,16340,2238


In [12]:
# Group data by state
state_groups = df.groupby('state')

# Calculate total count, killed count, and injured count
statesdf = state_groups.agg(
    total=('state', 'count'),  # Count all rows in each state group
    killed=('n_killed', 'sum'),  # Sum the 'killed' column values in each state group
    injured=('n_injured', 'sum'),  # Sum the 'injured' column values in each state group
    males=('Males', 'sum'),
    females=('Females', 'sum'),
)

# Reset index to include state as a column
statesdf = statesdf.reset_index()

In [13]:
state_to_code = {'District of Columbia' : 'dc','Mississippi': 'MS', 'Oklahoma': 'OK', 'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI', 'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX', 'Nevada': 'NV', 'Maine': 'ME'}
statesdf['state_code'] = statesdf['state'].apply(lambda x : state_to_code[x])

In [14]:
# Group data by state and year
state_year_groups = df.groupby(['state', 'year'])

# Calculate total count for each state and year
state_year_counts = state_year_groups.size().reset_index(name='year_count')

# Merge state_year_counts with statesdf
statesdf = statesdf.merge(state_year_counts, on=['state'], how='left')

# Fill missing year counts with 0
statesdf['year_count'] = statesdf['year_count'].fillna(0).astype(int)


In [15]:
statesdf = statesdf[(statesdf['year'] != 2013) & (statesdf['year'] != 2018)]
statesdf

,state,total,killed,injured,males,females,state_code,year,year_count
1,Alabama,5471,1880,2998,6990,1178,AL,2014,1318
2,Alabama,5471,1880,2998,6990,1178,AL,2015,1034
3,Alabama,5471,1880,2998,6990,1178,AL,2016,1309
4,Alabama,5471,1880,2998,6990,1178,AL,2017,1461
6,Alaska,1349,267,325,1428,238,AK,2014,146
...,...,...,...,...,...,...,...,...,...
285,Wisconsin,4787,862,2117,5098,801,WI,2017,1369
287,Wyoming,494,73,52,255,54,WY,2014,64
288,Wyoming,494,73,52,255,54,WY,2015,198
289,Wyoming,494,73,52,255,54,WY,2016,127


In [16]:
fig = go.Figure(data=go.Choropleth(
    locations=statesdf['state_code'], # Spatial coordinates
    z = statesdf['total'].astype(int), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Ylorrd',
    colorbar_title = "Millions USD",
))
fig.update_layout(
    title_text = '2011 US Agriculture Exports by State',
    geo = dict(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor='rgb(137, 207, 240)',),
)

In [17]:
# Group data by year
year_groups = df.groupby('year')

# Calculate counts (modify based on actual column names)
killed_counts = year_groups['n_killed'].sum()
injured_counts = year_groups['n_injured'].sum()

In [18]:
# Combine counts into a DataFrame (optional)
year_data = pd.DataFrame({'year': killed_counts.index, 'killed': killed_counts.values, 'injured': injured_counts.values})
year_data 


,year,killed,injured
0,2013,317,979
1,2014,12557,23002
2,2015,13484,26967
3,2016,15066,30580
4,2017,15511,30703
5,2018,3533,6171


In [19]:
if not year_data.empty:  # Check if data exists for the filtered year
    st.bar_chart(year_data, x='year', y=['killed', 'injured'])
else:
    st.write("No data available for the selected year.")

2024-05-06 19:27:49.098 
  command:

    streamlit run c:\Users\jakeh\anaconda3\envs\venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [20]:
df['participant_age_group'] = df['participant_age_group'].fillna("0::Zero")
df['Age'] = df['participant_age_group'].apply(lambda x: separate(x))
df['Child 0-11'] = df['Age'].apply(lambda x: x.count('Child 0-11'))
df['Teen 12-17'] = df['Age'].apply(lambda x: x.count('Teen 12-17'))
df['Adult 18+'] = df['Age'].apply(lambda x: x.count('Adult 18+'))

In [21]:
df = df[(df['year'] != 2013) & (df['year'] != 2018)]

In [22]:
age_group = df[['year', 'Child 0-11', 'Teen 12-17', 'Adult 18+']].groupby('year').sum()

In [23]:
age_group

,Child 0-11,Teen 12-17,Adult 18+
year,,,
2014,1280,4943,69285
2015,1086,5602,67004
2016,968,6307,74415
2017,905,6670,73047


In [26]:
# Group data by month and sum up the counts for each age group
age_group = df[['month', 'Child 0-11', 'Teen 12-17', 'Adult 18+']].groupby('month').sum()

# Define colors for each age group
colors = {
    'Child 0-11': 'blue',
    'Teen 12-17': 'green',
    'Adult 18+': 'red'
}

# Create traces for each age group
traces = []
for column in age_group.columns:
    trace = go.Scatter(x=age_group.index, y=age_group[column], mode='lines+markers', name=column,
                       line=dict(color=colors[column]))
    traces.append(trace)

# Create layout
layout = go.Layout(title='Gun Violence Incidents by Age Group Over the Months',
                   xaxis=dict(title='Month'),
                   yaxis=dict(title='Counts'))

# Create figure
fig = go.Figure(data=traces, layout=layout)
fig

In [40]:
def count_individual_incidents(incident_str):
    if isinstance(incident_str, str):  # Check if the value is a non-null string
        incidents = incident_str.split('||')
        incident_counts = {}
        for incident_entry in incidents:
            if incident_entry in incident_counts:
                incident_counts[incident_entry] += 1
            else:
                incident_counts[incident_entry] = 1
        return incident_counts
    else:
        return {}

In [42]:
# Initialize dictionary to hold the counts for all incidents
total_incident_counts = {}

# Iterate over each entry in the data
for entry in df['incident_characteristics']:
    individual_incident_counts = count_individual_incidents(entry)
    # Update total counts with counts from current entry
    for incident, count in individual_incident_counts.items():
        if incident in total_incident_counts:
            total_incident_counts[incident] += count
        else:
            total_incident_counts[incident] = count

# Print total counts for each incident
for incident, count in total_incident_counts.items():
    print("Incident:", incident, "- Count:", count)

Incident: Shots Fired - No Injuries - Count: 32945
Incident: Officer Involved Incident - Count: 16601
Incident: Shot - Dead (murder, accidental, suicide) - Count: 49636
Incident: Shot - Wounded/Injured - Count: 87975
Incident: Drive-by (car to street, car to car) - Count: 12689
Incident: Armed robbery with injury/death and/or evidence of DGU found - Count: 18684
Incident: Implied Weapon - Count: 1168
Incident: Home Invasion - Count: 9918
Incident: Home Invasion - Resident injured - Count: 2739
Incident: Pistol-whipping - Count: 3508
Incident: Accidental Shooting - Count: 7653
Incident: Accidental Shooting - Injury - Count: 4994
Incident: Cleaning gun - Count: 466
Incident: Brandishing/flourishing/open carry/lost/found - Count: 17345
Incident: Officer Involved Shooting - subject/suspect/perpetrator shot - Count: 3372
Incident: Possession (gun(s) found during commission of other crimes) - Count: 28831
Incident: Non-Shooting Incident - Count: 41309
Incident: Drug involvement - Count: 1565

In [45]:
# Sort the total incident counts in descending order
sorted_incident_counts = sorted(total_incident_counts.items(), key=lambda x: x[1], reverse=True)

# Print the top 3 most common incidents
print("Top 5 most common incidents:")
for i, (incident, count) in enumerate(sorted_incident_counts[:5], 1):
    print(f"{i}. {incident}: {count} occurrences")


Top 5 most common incidents:
1. Shot - Wounded/Injured: 87975 occurrences
2. Shot - Dead (murder, accidental, suicide): 49636 occurrences
3. Non-Shooting Incident: 41309 occurrences
4. Shots Fired - No Injuries: 32945 occurrences
5. Possession (gun(s) found during commission of other crimes): 28831 occurrences


In [ ]:
def count_individual_ages(age_str):
    if isinstance(age_str, str):  # Check if the value is a non-null string
        ages = age_str.split('||')
        age_counts = {}
        for age_entry in ages:
            parts = age_entry.split("::")
            if len(parts) == 2:
                index, age = parts
                age = int(age)  # Convert age to integer
                if age in age_counts:
                    age_counts[age] += 1
                else:
                    age_counts[age] = 1
        return age_counts
    else:
        return {}  # Return an empty dictionary if the value is NaN

In [ ]:
# Initialize dictionary to hold the counts for all ages
total_age_counts = {}

# Iterate over each entry in the data
for entry in df['participant_age']:
    individual_age_counts = count_individual_ages(entry)
    # Update total counts with counts from current entry
    for age, count in individual_age_counts.items():
        if age in total_age_counts:
            total_age_counts[age] += count
        else:
            total_age_counts[age] = count

# Print total counts for each age
for age, count in total_age_counts.items():
    print("Age:", age, "Count:", count)